<a href="https://colab.research.google.com/github/Tiberu2611/BaiTapPython/blob/main/Do_an_tksd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Thư Viện Cho Suy Diễn Thống Kê

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import statsmodels
import statsmodels.api as sm
from statsmodels.formula.api import ols
import scipy  
from scipy import stats
import statsmodels.stats.multicomp as mc
from google.colab import drive
from scipy.stats import f_oneway

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Đọc tập dữ liệu Diet_R.csv

In [3]:
drive.mount('/content/drive')
df = pd.read_csv('drive/My Drive/Diet_R.csv')

Mounted at /content/drive


Hiển thị các thông số cơ bản của tập dữ liệu

In [4]:
df.head()

,Person,gender,Age,Height,pre.weight,Diet,weight6weeks
0,25,,41,171,60,2,60.0
1,26,,32,174,103,2,103.0
2,1,0,22,159,58,1,54.2
3,2,0,46,192,60,1,54.0
4,3,0,55,170,64,1,63.3
...,...,...,...,...,...,...,...
73,74,1,35,183,83,3,80.2
74,75,1,49,177,84,3,79.9
75,76,1,28,164,85,3,79.7
76,77,1,40,167,87,3,77.8


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78 entries, 0 to 77
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Person        78 non-null     int64  
 1   gender        78 non-null     object 
 2   Age           78 non-null     int64  
 3   Height        78 non-null     int64  
 4   pre.weight    78 non-null     int64  
 5   Diet          78 non-null     int64  
 6   weight6weeks  78 non-null     float64
dtypes: float64(1), int64(5), object(1)
memory usage: 4.4+ KB


**Kiểm tra các giả thuyết để xem có giảm cân được hay không**

Phát biểu giả thiết

$ H_{0}:$ Cân nặng trước khi giảm cân và sau khi giảm cân như nhau.
$$μ_{0} = μ_{1} $$
$ H_{1}:$ Cân nặng trước khi giảm cân và sau khi giảm cân khác nhau.
$$μ_{0} \neq μ_{1} $$


In [ ]:
ttest_score, p=stats.ttest_ind(df["pre.weight"], df["weight6weeks"])
print('starts = ',ttest_score , 'p value ', p)
if (p > 0.05):
    print("Không có bằng chứng để bác bỏ H0 ")
    print("Cân nặng trước và sau khi giảm cân không thay đổi")
else:
    print('Đã có đủ bằng chứng để bác bỏ H0')
    print("Cân nặng trước và sau khi giảm cân có sự thay đổi")

starts =  2.720972750635475 p value  0.0072584136689251115
Đã có đủ bằng chứng để bác bỏ H0
Cân nặng trước và sau khi giảm cân có sự thay đổi


**Thống kê các phương pháp giảm cân xem xét sự thay đổi của nó**

In [ ]:
df['lostweight'] = df['pre.weight'] - df['weight6weeks']
print(df)

    Person gender  Age  Height  pre.weight  Diet  weight6weeks  lostweight
0       25          41     171          60     2          60.0         0.0
1       26          32     174         103     2         103.0         0.0
2        1      0   22     159          58     1          54.2         3.8
3        2      0   46     192          60     1          54.0         6.0
4        3      0   55     170          64     1          63.3         0.7
..     ...    ...  ...     ...         ...   ...           ...         ...
73      74      1   35     183          83     3          80.2         2.8
74      75      1   49     177          84     3          79.9         4.1
75      76      1   28     164          85     3          79.7         5.3
76      77      1   40     167          87     3          77.8         9.2
77      78      1   51     175          88     3          81.9         6.1

[78 rows x 8 columns]


**Phân nhóm giá trị Diet để xem giá trị cân nặng đã giảm theo giá trị từng nhóm**

In [ ]:
Dt = df[['Diet', 'lostweight']]
diet = df.groupby('Diet').groups
wl_1 = Dt['lostweight'][diet[1]]
wl_1.name = 'Diet 1'
wl_2 = Dt['lostweight'][diet[2]]
wl_2.name = 'Diet 2'
wl_3 = Dt['lostweight'][diet[3]]
wl_3.name = 'Diet 3'
print(wl_1.describe(include=['lostweight']))
print(wl_2.describe(include=['lostweight']))
print(wl_3.describe(include=['lostweight']))

count    24.000000
mean      3.300000
std       2.240148
min      -0.600000
25%       1.975000
50%       3.050000
75%       3.950000
max       9.000000
Name: Diet 1, dtype: float64
count    27.000000
mean      3.025926
std       2.523367
min      -2.100000
25%       1.700000
50%       3.300000
75%       4.500000
max       7.900000
Name: Diet 2, dtype: float64
count    27.000000
mean      5.148148
std       2.395568
min       0.500000
25%       3.450000
50%       5.400000
75%       7.000000
max       9.200000
Name: Diet 3, dtype: float64


**Kiểm định ANOVA một chiều**

 sử dụng Shapiro-Wilk test

Phát biểu giả thiết:

$H_0: \text{Diet 1 là dữ liệu tuân theo phân phối chuẩn}$

$H_1: \text{Diet 1 không phải là dữ liệu tuân theo phân phối chuẩn}$

In [ ]:
start_1, p_value_1 = stats.shapiro(wl_1)
print('statistic of Diet 1 = ',start_1)
print('pvalue of Diet 1 = ',p_value_1)
if (p_value_1 > 0.05):
    print("Không có bằng chứng để bác bỏ H0: ")
    print('Đủ điều kiện để phân tích tiếp')
else:
    print('Đã có đủ bằng chứng để bác bỏ H0')
    print('Diet 1 không phải là dữ liệu tuân theo phân phối chuẩn')

statistic of Diet 1 =  0.9255314469337463
pvalue of Diet 1 =  0.07748671621084213
Không có bằng chứng để bác bỏ H0: 
Đủ điều kiện để phân tích tiếp


Phát biểu giả thiết:

$H_0: \text{Diet 2 là dữ liệu tuân theo phân phối chuẩn}$

$H_1: \text{Diet 2 không phải là dữ liệu tuân theo phân phối chuẩn}$

In [ ]:
start_2, p_value_2 = stats.shapiro(wl_2)
print('statistic of Diet 2 = ',start_2)
print('pvalue of Diet 2 = ',p_value_2)
if (p_value_2 > 0.05):
    print("Không có bằng chứng để bác bỏ H0: ")
    print('Đủ điều kiện để phân tích tiếp')
else:
    print('Đã có đủ bằng chứng để bác bỏ H0')
    print('Diet 2 không phải là dữ liệu tuân theo phân phối chuẩn')

statistic of Diet 2 =  0.9855913519859314
pvalue of Diet 2 =  0.961176335811615
Không có bằng chứng để bác bỏ H0: 
Đủ điều kiện để phân tích tiếp


Phát biểu giả thiết:

$H_0: \text{Diet 3 là dữ liệu tuân theo phân phối chuẩn}$

$H_1: \text{Diet 3 không phải là dữ liệu tuân theo phân phối chuẩn}$

In [ ]:
start_3, p_value_3 = stats.shapiro(wl_3)
print('statistic of Diet 3 = ',start_3)
print('pvalue of Diet 3 = ',p_value_3)
if (p_value_3 > 0.05):
    print("Không có bằng chứng để bác bỏ H0: ")
    print('Đủ điều kiện để phân tích tiếp')
else:
    print('Đã có đủ bằng chứng để bác bỏ H0')
    print('Diet 3 không phải là dữ liệu tuân theo phân phối chuẩn')

statistic of Diet 3 =  0.9601252675056458
pvalue of Diet 3 =  0.37203603982925415
Không có bằng chứng để bác bỏ H0: 
Đủ điều kiện để phân tích tiếp


**Kiểm tra tính đồng nhất của các phương sai**

ta sử dụng Barlett test

In [ ]:
start_13, p_value_13 = stats.bartlett(wl_1, wl_2, wl_3)
print('statistic by Bartlett = ',start_13)
print('pvalue by Bartlett = ',p_value_13)
if (p_value_13 > 0.05):
    print('Không có bằng chứng để bác bỏ H0')
    print('Có giả thuyết chứng minh mẫu có phương sai giống nhau')
    print('Đủ điều kiện đê phân tích ANOVA')
else:
    print('Đã có đủ bằng chứng để bác bỏ giả thuyết Ho')
    print('Có ít nhất 2 feature có phương sai khác nhau')

statistic by Bartlett =  0.337454630479139
pvalue by Bartlett =  0.8447392194944549
Không có bằng chứng để bác bỏ H0
Điều này ngụ ý Các mẫu có phương sai giống nhau
Đủ điều kiện đê phân tích ANOVA


**Kiểm định ANOVA 1 chiều**

**Khả năng giảm cân có phụ thuộc vào cách thức giảm cân hay không ?**

Phát biểu giả thiết:

$H_{0}: \mu_{1} = \mu_{2} = \mu_{3}$

$H_{a}: \text{Có tối thiểu một cặp giá trị trung bình có giá trị khác nhau}$

In [ ]:
start, p_value = f_oneway(wl_1, wl_2, wl_3)
alpha = 0.05
print('Giá trị kiểm định: ',start)
print('p_value: ',p_value)
if (p < alpha):
    print('Có bằng chứng bác bỏ giả thuyết H0')
    print("Chứng minh rằng hiệu quả giữa các phương pháp có sự khác biệt")
else:
    print("Không đủ bằng chứng bác bỏ giả thuyết H0.")
    print("Chứng minh rằng hiệu quả giữa các phương pháp không có sự khác biệt")

Giá trị kiểm định:  6.197447453165349
p_value:  0.0032290142385893524
Có bằng chứng bác bỏ giả thuyết H0
Chứng minh rằng hiệu quả giữa các phương pháp có sự khác biệt
